
# SBS 3D Video Generation Pipeline

This notebook outlines the process of converting a monocular video into a side-by-side (SBS) 3D video.



## Setup and Preparation

Import necessary libraries and define the input video path.

Personally, I opt for the file structure `datasets/d{index}/[set of input/output folder for frames]`

In [ ]:
import timeit
import os
import subprocess

dataset = 0
# Define the path to the input video
dataset_directory = f'datasets/d{dataset}/'
input_video_path = dataset_directory + 'test.mp4'
final_video_filename = dataset_directory + 'test_SBS.mp4'

In [ ]:
# stable
import os
import cv2
import numpy as np
import concurrent.futures
from tqdm import tqdm

def calculate_disparity(depth_value, scale_factor):
    try:
        disparity = int(depth_value * scale_factor)
        return disparity
    except Exception as e:
        print(f"Exception in calculate_disparity for depth_value {depth_value}: {e}")
        return 0

def shift_pixels(image, depth_map, direction, scale_factor):
    try:
        print("Starting shift_pixels operation with handling for black pixels")

        # Calculate disparity map and apply scaling factor
        disparity_map = (depth_map.astype(np.float32) * scale_factor).astype(np.int32)

        # Get image dimensions
        height, width, _ = image.shape

        # Generate grid of x and y coordinates
        x_grid, y_grid = np.meshgrid(np.arange(width), np.arange(height))

        # Calculate new x coordinates after applying disparity shift
        new_x = x_grid + (disparity_map * direction)

        # Initialize shifted image with zeros (black)
        shifted_image = np.zeros_like(image)

        # Determine which of the new x coordinates are within image bounds
        in_bounds_mask = (new_x >= 0) & (new_x < width)

        # Use boolean indexing to select pixels within bounds and assign them to the new coordinates in the shifted image
        shifted_image[y_grid[in_bounds_mask], new_x[in_bounds_mask]] = image[y_grid[in_bounds_mask], x_grid[in_bounds_mask]]

        print("Completed shift_pixels operation with handling for black pixels")
        return shifted_image
    except Exception as e:
        print(f"Exception in shift_pixels: {e}")
        return np.zeros_like(image)


def fast_shift_pixels(image, depth_map, direction, scale_factor):
    try:
        print("Starting shift_pixels operation with disparity calculation")

        # Ensure depth_map is a 2D array if it's not already grayscale
        if depth_map.ndim == 3:
            depth_map = cv2.cvtColor(depth_map, cv2.COLOR_BGR2GRAY)

        # Calculate disparity as a 2D array
        disparity = (depth_map.astype(np.float32) * scale_factor).astype(np.int32)

        # Create an array of x-coordinates and add the disparity to shift
        # Note: We're limiting the shift to ensure it remains within image bounds
        x_coords = np.arange(image.shape[1])
        y_coords = np.arange(image.shape[0])[:, None]  # Making it a column vector for broadcasting

        # Calculate new x-coordinates after applying the disparity shift
        # Using np.clip to ensure the shifted coordinates remain within bounds
        shifted_x_coords = np.clip(x_coords + disparity * direction, 0, image.shape[1] - 1)

        # Use advanced indexing to create the shifted image
        shifted_image = image[y_coords, shifted_x_coords]

        print("Completed shift_pixels operation with disparity calculation")
        return shifted_image
    except Exception as e:
        print(f"Exception in shift_pixels: {e}")
        return np.zeros_like(image)




def process_image_pair(color_image_path, depth_image_path, output_dir, frame_number, scale_factor):
    try:
        print(f"Thread started for frame {frame_number}")

        color_image = cv2.imread(color_image_path)
        depth_map = cv2.imread(depth_image_path, cv2.IMREAD_UNCHANGED)

        if color_image is None or depth_map is None:
            print(f"Error: Missing image for frame {frame_number}")
            return

        # Assuming depth_map might not be in grayscale, convert if necessary
        if depth_map.ndim == 3:
            depth_map = cv2.cvtColor(depth_map, cv2.COLOR_BGR2GRAY)

        left_eye_image = shift_pixels(color_image, depth_map, 1, scale_factor)
        right_eye_image = shift_pixels(color_image, depth_map, -1, scale_factor)

        # Construct output paths
        left_eye_output_path = os.path.join(output_dir, f'leftEye{frame_number}.jpg')
        right_eye_output_path = os.path.join(output_dir, f'rightEye{frame_number}.jpg')

        # Save the shifted images
        cv2.imwrite(left_eye_output_path, left_eye_image)
        cv2.imwrite(right_eye_output_path, right_eye_image)

        print(f"Processed and saved images for frame {frame_number}")
    except Exception as e:
        print(f"Exception in process_image_pair for frame {frame_number}: {e}")


# Setup directories and process images
input_directory = "C:/Users/abahrema/Documents/Tools/sbs-generator/datasets/d0/rgbd_in"
output_directory = "C:/Users/abahrema/Documents/Tools/sbs-generator/datasets/d0/scratch_test"
print("Creating output directory if it doesn't exist.")
os.makedirs(output_directory, exist_ok=True)
print("Reading color images from the input directory.")

color_images = sorted([f for f in os.listdir(input_directory) if f.startswith('color')])
depth_images = sorted([f for f in os.listdir(input_directory) if f.startswith('depth')])

with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count() - 1) as executor:
    futures = []
    for color_image, depth_image in zip(color_images, depth_images):
        color_image_path = os.path.join(input_directory, color_image)
        depth_image_path = os.path.join(input_directory, depth_image)
        frame_number = color_image.split('color')[1].split('.')[0]
        futures.append(executor.submit(process_image_pair, color_image_path, depth_image_path, output_directory, frame_number, 0.1))

    # Progress tracking
    for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        pass

print("Image processing completed.")


In [ ]:
import concurrent.futures
import numpy as np
from tqdm import tqdm

# Function to sort an array in a separate thread
def sort_array(arr):
    print(f"Sorting: {arr}")
    sorted_arr = np.sort(arr)
    print(f"Sorted: {sorted_arr}")
    return sorted_arr

# Generating a list of random integer arrays
num_arrays = 10
array_size = 5
arrays = [np.random.randint(0, 100, array_size) for _ in range(num_arrays)]

print("Original arrays:")
for arr in arrays:
    print(arr)

# Using ThreadPoolExecutor to sort each array in a separate thread
sorted_arrays = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Creating a progress bar with tqdm
    with tqdm(total=num_arrays, desc="Sorting Arrays") as progress:
        # Submitting tasks to the executor
        future_to_array = {executor.submit(sort_array, arr): arr for arr in arrays}
        for future in concurrent.futures.as_completed(future_to_array):
            sorted_arrays.append(future.result())
            progress.update(1)

print("\nSorted arrays:")
for arr in sorted_arrays:
    print(arr)


In [ ]:
import os
import cv2
import numpy as np
import concurrent.futures
from tqdm import tqdm

def process_image_pair(color_image_filename):
    # Define the functions inline
    def calculate_disparity(depth_value, scale_factor):
        return int(depth_value * scale_factor)

    def fast_shift_pixels(image, depth_map, direction, scale_factor):
        # print("Starting fast_shift_pixels")
        # Calculate disparity for the entire depth map in a vectorized manner
        disparity = (depth_map * scale_factor).astype(np.int32)
        # print("Disparity calculated")        
        # Prepare an array of indices for x-coordinates
        height, width = image.shape[:2]
        x_indices = np.tile(np.arange(width), (height, 1))        
        # Apply disparity shift
        new_x_indices = x_indices + disparity * direction        
        # Ensure new indices are within bounds
        new_x_indices = np.clip(new_x_indices, 0, width - 1)        
        # Generate y-coordinates grid
        y_indices = np.repeat(np.arange(height)[:, np.newaxis], width, axis=1)
        # Use advanced indexing to shift pixels
        shifted_image = image[y_indices, new_x_indices]
        # print("Image shifted")        
        return shifted_image

    def shift_pixels(image, depth_map, direction, scale_factor):
        try:
            print(f"Starting shift_pixels on {image.shape}")
            # Perform a very basic operation
            shifted_image = np.zeros_like(image)
            # print("Basic operation done, proceeding to disparity")
            
            # Now add the disparity calculation
            disparity = (depth_map * scale_factor).astype(np.int32)
            # print(f"Disparity calculated: {disparity}")
    
            # If this prints, the problem is further down
            # print("Disparity calculation done, proceeding to pixel shift")
    
            # disparity = int(depth_value * scale_factor) # calculate_disparity(depth_map[y, x], scale_factor)
            for y in range(image.shape[0]):
                for x in range(image.shape[1]):
                    disparity = calculate_disparity(depth_map[y, x], scale_factor)
                    new_x = int(x + disparity * direction)
                    if 0 <= new_x < image.shape[1]:
                        shifted_image[y, new_x] = image[y, x]
            
            new_x = x + disparity * direction
            if 0 <= new_x < image.shape[1]:
                shifted_image[y, new_x] = image[y, x]
            
            return shifted_image
        except Exception as e:
            print(f"Exception in shift_pixels: {e}")
            import traceback
            traceback.print_exc()


    # Continue with the existing logic
    depth_image_filename = color_image_filename.replace('color', 'depth').replace('.jpg', '.png')
    color_image_path = os.path.join(input_directory, color_image_filename)
    depth_image_path = os.path.join(input_directory, depth_image_filename)

    print(f"Processing pair: {color_image_filename} & {depth_image_filename}")

    color_image = cv2.imread(color_image_path)
    depth_map = cv2.imread(depth_image_path, cv2.IMREAD_UNCHANGED)  # Reading depth map as PNG

    if color_image is None or depth_map is None:
        print(f"Error: Missing image for {color_image_filename}")
        return

    shifted_image = fast_shift_pixels(color_image, depth_map, 1, 0.05)

    print("shifted")
    output_filename = f"shifted_{color_image_filename}"
    output_path = os.path.join(output_directory, output_filename)

    cv2.imwrite(output_path, shifted_image)
    print(f"Processed and saved: {output_filename}")

# Directories
input_directory = "C:/Users/abahrema/Documents/Tools/sbs-generator/datasets/d0/rgbd_in"
output_directory = "C:/Users/abahrema/Documents/Tools/sbs-generator/datasets/d0/scratch_test"
print("Creating output directory if it doesn't exist.")
os.makedirs(output_directory, exist_ok=True)
print("Reading color images from the input directory.")
color_images = sorted([f for f in os.listdir(input_directory) if f.endswith('.jpg') and 'color' in f])

print(f"Found {len(color_images)} color images to process.")
num_cores = os.cpu_count()
print(f"Num cores {num_cores}")

with concurrent.futures.ThreadPoolExecutor(max_workers=num_cores-1) as executor:
    with tqdm(total=len(color_images), desc="Processing Image Pairs") as progress:
        futures = [executor.submit(process_image_pair, color_image_filename) for color_image_filename in color_images]
        for future in concurrent.futures.as_completed(futures):
            progress.update(1)

print("Image processing completed.")


In [ ]:
import os
import cv2
import numpy as np
import concurrent.futures
from tqdm import tqdm

def process_image_pair(color_image_filename):
    def calculate_disparity(depth_value, scale_factor):
        try:
            disparity = int(depth_value) * scale_factor
            # print(f"Calculating disparity: Depth value {depth_value}, Scale factor {scale_factor}, Disparity {disparity}")
            return disparity
        except Exception as e:
            print(f"Exception in calculate_disparity: {e}")
            return 0
    
    def shift_pixels(image, depth_map, direction, scale_factor):
        try:
            print("Starting shift_pixels operation")
            shifted_image = np.zeros_like(image)
            for y in range(image.shape[0]):
                for x in range(image.shape[1]):
                    disparity = calculate_disparity(depth_map[y, x], scale_factor)
                    new_x = int(x + disparity * direction)
                    if 0 <= new_x < image.shape[1]:
                        shifted_image[y, new_x] = image[y, x]
                    # if x % 100 == 0 and y % 100 == 0:  # Adjust the frequency of this print as needed
                    #     print(f"At pixel ({y}, {x}), Disparity: {disparity}, New X: {new_x}")
            # print("Completed shift_pixels operation")
            return shifted_image
        except Exception as e:
            print(f"Exception in shift_pixels: {e}")
            import traceback
            traceback.print_exc()
            return np.zeros_like(image)


    depth_image_filename = color_image_filename.replace('color', 'depth').replace('.jpg', '.png')
    color_image_path = os.path.join(input_directory, color_image_filename)
    depth_image_path = os.path.join(input_directory, depth_image_filename)

    print(f"Processing pair: {color_image_filename} & {depth_image_filename}")

    color_image = cv2.imread(color_image_path)
    depth_map = cv2.imread(depth_image_path, cv2.IMREAD_UNCHANGED)

    if color_image is None or depth_map is None:
        print(f"Error: Missing image for {color_image_filename}")
        return

    # Convert depth map to grayscale
    depth_map = cv2.cvtColor(depth_map, cv2.COLOR_BGR2GRAY) if len(depth_map.shape) == 3 else depth_map

    shifted_image = shift_pixels(color_image, depth_map, 1, 0.05)

    print("Shifted")
    output_filename = f"shifted_{color_image_filename}"
    output_path = os.path.join(output_directory, output_filename)

    cv2.imwrite(output_path, shifted_image)
    print(f"Processed and saved: {output_filename}")

# Directories
input_directory = "C:/Users/abahrema/Documents/Tools/sbs-generator/datasets/d0/rgbd_in"
output_directory = "C:/Users/abahrema/Documents/Tools/sbs-generator/datasets/d0/scratch_test"

print("Creating output directory if it doesn't exist.")
os.makedirs(output_directory, exist_ok=True)

print("Reading color images from the input directory.")
color_images = sorted([f for f in os.listdir(input_directory) if f.endswith('.jpg') and 'color' in f])

print(f"Found {len(color_images)} color images to process.")
num_cores = os.cpu_count()
print(f"Num cores {num_cores}")

with concurrent.futures.ThreadPoolExecutor(max_workers=num_cores-1) as executor:
    with tqdm(total=len(color_images), desc="Processing Image Pairs") as progress:
        futures = [executor.submit(process_image_pair, color_image_filename) for color_image_filename in color_images]
        for future in concurrent.futures.as_completed(futures):
            progress.update(1)

print("Image processing completed.")


## Import and Setup Depth Anything Project

Run depth anything model for video input, specify if wishing to include depth model locally

In [ ]:
local_path = False
local_path_directory = "your_local_directory"
current_working_directory = os.getcwd()

In [ ]:
if local_path:
    local_path_directory = r"depth_models"
    # Clone the repository
    os.system("git clone https://github.com/LiheYoung/Depth-Anything")
    # Change directory to the cloned repository
    os.chdir("Depth-Anything")
    # Create a Conda environment named 'depth-anything' with Python 3.11
    os.system("conda create -n depth-anything python=3.11")
    # Activate the Conda environment
    os.system("conda activate depth-anything")
    # Install PyTorch and other dependencies
    os.system("conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia")
    # Install additional Python packages from requirements.txt
    os.system("pip install -r requirements.txt")
    # Print a message to indicate successful setup
    print("Project setup complete.")

In [ ]:
# if not local path, specify remote path for depth-anything
remote_path = r"..\Depth-Anything"
if not local_path: local_path_directory = remote_path
print(f"Path set {local_path_directory}")

In [ ]:
def checkPath(path):
    if os.path.exists(path): print(f"The file '{path}' exists.")
    else: print(f"The file '{path}' does not exist.")

In [ ]:
rgbd_frames = dataset_directory + 'rgbd_in/'
os.makedirs(rgbd_frames, exist_ok=True)

print(f"dataset directory {dataset_directory}\n video dire {input_video_path}\n outdir {dataset_directory}")
print(f"local path {local_path_directory}")

# Check if the file exists
file_path = os.path.join(local_path_directory, "run_depth_only.py")

# Get the current working directory
current_working_directory = os.getcwd()

# Define the full paths to input video and dataset directory
input_video_path_full = os.path.join(current_working_directory, input_video_path)
dataset_directory_full = os.path.join(current_working_directory, dataset_directory)
checkPath(input_video_path_full)
checkPath(dataset_directory_full)


os.chdir(local_path_directory)
!python {local_path_directory}/run_depth_only.py --encoder vitl --video-path {input_video_path_full} --outdir {dataset_directory_full}


## Extract Frames from Video

Use ffmpeg to extract frames from the input color video and depth video.


In [ ]:
# switch directory back
output_depth_video = dataset_directory + "test_video_depth.mp4"
os.chdir(current_working_directory)
print(f"Changed directory to {current_working_directory}")

In [ ]:
# Create directory if non-existent
output_frames_path = dataset_directory + 'rgbd_in/frame%d.jpg'
output_dir = os.path.dirname(output_frames_path)
os.makedirs(output_dir, exist_ok=True)
print(f"output_frames_path: {output_frames_path}, output_dir: {output_dir}")
print(f"video {input_video_path}")
# execute ffmpeg command for color
!ffmpeg -i {input_video_path} -q:v 2 {output_frames_path} 
output_frames_path = dataset_directory + 'rgbd_in/frame%d.png' # for depth
!ffmpeg -i {output_depth_video} -q:v 2 {output_frames_path}



## Image Preprocessing

Rename and pair color and depth images as needed. Run the script or run the function inside notebook.


In [ ]:
rgbd_frames = dataset_directory + 'rgbd_in/'
# Example for renaming images (adjust according to your script)
!python sbs_rename_directory.py {rgbd_frames}

In [ ]:
import os
import re

def get_frame_number(filename):
    match = re.search(r"frame(\d+)_", filename)
    if match:
        return int(match.group(1))
    else:
        raise ValueError(f"Invalid filename format: {filename}")
        
def rename_files(source_dir):
    os.makedirs(source_dir, exist_ok=True)

    # Process color images
    color_files = sorted([f for f in os.listdir(source_dir) if f.startswith("frame") and f.endswith(".jpg")], 
                         key=lambda x: int(x.split("frame")[1].split(".")[0]))
    counter = 1
    for filename in color_files:
        new_name = f"color{counter}.jpg"
        os.rename(os.path.join(source_dir, filename), os.path.join(source_dir, new_name))
        counter += 1
    print(f"Renamed {counter} color files in {source_dir}.")

    # Process depth images
    depth_files = sorted([f for f in os.listdir(source_dir) if f.startswith("frame") and f.endswith(".png")], 
                         key=lambda x: int(x.split("frame")[1].split(".")[0]))
    counter = 1
    for filename in depth_files:
        new_name = f"depth{counter}.png"
        os.rename(os.path.join(source_dir, filename), os.path.join(source_dir, new_name))
        counter += 1
    print(f"Renamed {counter} depth files in {source_dir}.")
    
source_dir = dataset_directory + "rgbd_in/"
rename_files(source_dir)


## Generate Stereo Views

Run the script to generate left and right eye views or run the function inside notebook.


In [ ]:
stereo_input_dir =  dataset_directory + "rgbd_in/"
stereo_output_dir = dataset_directory + "stereo_out_frames/"
os.makedirs(stereo_input_dir, exist_ok=True)
os.makedirs(stereo_output_dir, exist_ok=True)

!python sbs_generate_stereoviews.py {stereo_input_dir} {stereo_output_dir}

In [ ]:
%time

import cv2
import numpy as np
import os

def process_images(input_dir, output_dir, scale_factor):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    color_images = sorted([f for f in os.listdir(input_dir) if f.startswith('color')])
    depth_images = sorted([f for f in os.listdir(input_dir) if f.startswith('depth')])

    for color_image_path, depth_image_path in zip(color_images, depth_images):
        color_image = cv2.imread(os.path.join(input_dir, color_image_path))
        depth_map = cv2.imread(os.path.join(input_dir, depth_image_path), cv2.IMREAD_GRAYSCALE)

        if color_image is None:
            print(f"Error: Color image not found at {os.path.join(input_dir, color_image_path)}")
            continue

        if depth_map is None:
            print(f"Error: Depth map not found at {os.path.join(input_dir, depth_image_path)}")
            continue

        # Function to shift pixels based on depth map
        def shift_pixels(image, depth_map, direction):
            shifted_image = np.zeros_like(image)
            for y in range(image.shape[0]):
                for x in range(image.shape[1]):
                    disparity = calculate_disparity(depth_map[y, x])
                    new_x = x + disparity * direction
                    if 0 <= new_x < image.shape[1]:
                        shifted_image[y, new_x] = image[y, x]
            return shifted_image

        # Calculate disparity (example function, adjust as needed)
        def calculate_disparity(depth_value):
            # Simple linear mapping, adjust the scale factor as needed
            return int(depth_value * scale_factor)

        # Create left and right eye images
        left_eye_image = shift_pixels(color_image, depth_map, 1)
        right_eye_image = shift_pixels(color_image, depth_map, -1)

        frame_number = color_image_path.split('color')[1].split('.')[0]
        
        if not os.path.exists(os.path.join(output_dir,'leftEye')):
            os.makedirs(os.path.join(output_dir,'leftEye'))
        if not os.path.exists(os.path.join(output_dir,'rightEye')):
            os.makedirs(os.path.join(output_dir,'rightEye'))
        
        left_eye_output_path = os.path.join(output_dir, f'leftEye/leftEye{frame_number}.jpg')
        right_eye_output_path = os.path.join(output_dir, f'rightEye/rightEye{frame_number}.jpg')

        # Save the left and right eye images
        cv2.imwrite(left_eye_output_path, left_eye_image)
        cv2.imwrite(right_eye_output_path, right_eye_image)

        print(f"Processed frame {frame_number}.")

# Example usage
stereo_input_dir =  dataset_directory + "rgbd_in/"
stereo_output_dir = dataset_directory + "stereo_out_frames/"
os.makedirs(stereo_input_dir, exist_ok=True)
os.makedirs(stereo_output_dir, exist_ok=True)
process_images(stereo_input_dir, stereo_output_dir, 0.05)

In [ ]:
%time

import cv2
import numpy as np
import os
import concurrent.futures
from tqdm import tqdm

def calculate_disparity(depth_value, scale_factor):
    return int(depth_value * scale_factor)

def shift_pixels(image, depth_map, direction, scale_factor):
    shifted_image = np.zeros_like(image)
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            disparity = calculate_disparity(depth_map[y, x], scale_factor)
            new_x = x + disparity * direction
            if 0 <= new_x < image.shape[1]:
                shifted_image[y, new_x] = image[y, x]
    return shifted_image

def process_single_image_pair(args):
    color_image_path, depth_image_path, input_dir, output_dir, scale_factor = args
    color_image = cv2.imread(os.path.join(input_dir, color_image_path))
    depth_map = cv2.imread(os.path.join(input_dir, depth_image_path), cv2.IMREAD_GRAYSCALE)

    if color_image is None or depth_map is None:
        return f"Error: Image not found for {color_image_path} or {depth_image_path}"

    left_eye_image = shift_pixels(color_image, depth_map, 1, scale_factor)
    right_eye_image = shift_pixels(color_image, depth_map, -1, scale_factor)

    frame_number = color_image_path.split('color')[1].split('.')[0]

    left_eye_output_path = os.path.join(output_dir, 'leftEye', f'leftEye{frame_number}.jpg')
    right_eye_output_path = os.path.join(output_dir, 'rightEye', f'rightEye{frame_number}.jpg')

    cv2.imwrite(left_eye_output_path, left_eye_image)
    cv2.imwrite(right_eye_output_path, right_eye_image)

    return f"Processed frame {frame_number}"

def process_images(input_dir, output_dir, scale_factor):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    color_images = sorted([f for f in os.listdir(input_dir) if f.startswith('color')])
    depth_images = sorted([f for f in os.listdir(input_dir) if f.startswith('depth')])

    image_pairs = list(zip(color_images, depth_images))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        args = [(color_image, depth_image, input_dir, output_dir, scale_factor) for color_image, depth_image in image_pairs]
        results = list(tqdm(executor.map(process_single_image_pair, args), total=len(args)))

    for result in results:
        if result:
            print(result)

stereo_input_dir = os.path.join(dataset_directory, "rgbd_in/")
stereo_output_dir = os.path.join(dataset_directory, "stereo_out_frames/")
os.makedirs(stereo_input_dir, exist_ok=True)
os.makedirs(stereo_output_dir, exist_ok=True)
print("About to process")
process_images(stereo_input_dir, stereo_output_dir, 0.05)


## Inpainting Process

Run the script for inpainting left and right eye images or run the function inside notebook.


In [ ]:
stereo_output_dir = dataset_directory + "stereo_out_frames/"
stereo_postprocess_dir = dataset_directory + "stereo_postprocess_frames/"
os.makedirs(stereo_output_dir, exist_ok=True)
os.makedirs(stereo_postprocess_dir, exist_ok=True)

!python sbs_inpaint_stereoviews.py {stereo_output_dir} {stereo_postprocess_dir}

In [ ]:
import cv2
import numpy as np
import os

def create_mask_for_black_streaks(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Use adaptive thresholding to better capture the black streaks
    mask = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 3, 8)
    
    # Dilate the mask to include the edges of the black streaks
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=1)
    
    return mask

def inpaint_black_streaks(image, mask):
    # Inpaint the black streaks in the image
    inpainted_image = cv2.inpaint(image, mask, 5, cv2.INPAINT_TELEA)
    
    return inpainted_image

def process_images(input_dir, output_dir, save_masks=False):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if not os.path.exists(os.path.join(output_dir,'leftEye')):
        os.makedirs(os.path.join(output_dir,'leftEye'))
    if not os.path.exists(os.path.join(output_dir,'rightEye')):
        os.makedirs(os.path.join(output_dir,'rightEye'))
    
    if save_masks:
        if not os.path.exists(os.path.join(output_dir,'leftEyeMask')):
            os.makedirs(os.path.join(output_dir,'leftEyeMask'))
        if not os.path.exists(os.path.join(output_dir,'rightEyeMask')):
            os.makedirs(os.path.join(output_dir,'rightEyeMask'))

    left_path = input_dir + "leftEye/"
    right_path = input_dir + "rightEye/"
    left_eye_images = sorted([f for f in os.listdir(left_path) if f.startswith('leftEye')])
    right_eye_images = sorted([f for f in os.listdir(right_path) if f.startswith('rightEye')])

    for left_eye_image_path, right_eye_image_path in zip(left_eye_images, right_eye_images):
        left_eye_image = cv2.imread(os.path.join(left_path, left_eye_image_path))
        right_eye_image = cv2.imread(os.path.join(right_path, right_eye_image_path))

        if left_eye_image is None or right_eye_image is None:
            print(f"Error: Image not found at {os.path.join(input_dir, left_eye_image_path)} or {os.path.join(input_dir, right_eye_image_path)}")
            continue

        # Create masks for the black streaks in both left and right eye images
        left_eye_mask = create_mask_for_black_streaks(left_eye_image)
        right_eye_mask = create_mask_for_black_streaks(right_eye_image)

        # Inpaint the black streaks in both left and right eye images
        left_eye_post = inpaint_black_streaks(left_eye_image, left_eye_mask)
        right_eye_post = inpaint_black_streaks(right_eye_image, right_eye_mask)

        frame_number = left_eye_image_path.split('leftEye')[1].split('.')[0]
        left_eye_post_output_path = os.path.join(output_dir + "leftEye/", f'leftEyePost{frame_number}.jpg')
        right_eye_post_output_path = os.path.join(output_dir + "rightEye/", f'rightEyePost{frame_number}.jpg')
        # Save the processed images and masks
        cv2.imwrite(left_eye_post_output_path, left_eye_post)
        cv2.imwrite(right_eye_post_output_path, right_eye_post)
        
        if (save_masks):
            left_eye_mask_output_path = os.path.join(output_dir + "leftEyeMask/", f'leftEyeMask{frame_number}.jpg')
            right_eye_mask_output_path = os.path.join(output_dir + "rightEyeMask/", f'rightEyeMask{frame_number}.jpg')
            cv2.imwrite(left_eye_mask_output_path, left_eye_mask)
            cv2.imwrite(right_eye_mask_output_path, right_eye_mask)

        print(f"Processed frame {frame_number}.")

                          
stereo_output_dir = dataset_directory + "stereo_out_frames/"
stereo_postprocess_dir = dataset_directory + "stereo_postprocess_frames/"
os.makedirs(stereo_output_dir, exist_ok=True)
os.makedirs(stereo_postprocess_dir, exist_ok=True)                          
# Example usage
process_images(stereo_output_dir, stereo_postprocess_dir)



## Create Videos from Images

Use ffmpeg to create left and right eye videos. 

**Note** what the appropriate frame rate should be based on your input video.


In [ ]:
left_eye_dir = dataset_directory + "stereo_postprocess_frames/" + "leftEye/"
right_eye_dir = dataset_directory + "stereo_postprocess_frames/" + "rightEye/"
os.makedirs(left_eye_dir, exist_ok=True)
os.makedirs(right_eye_dir, exist_ok=True)
left_eye_dir += "leftEyePost%d.jpg"
right_eye_dir += "rightEyePost%d.jpg"

left_eye_vid = dataset_directory + "left_eye.mp4"
right_eye_vid = dataset_directory + "right_eye.mp4"


!ffmpeg -framerate 24 -i {left_eye_dir} -c:v libx264 -pix_fmt yuv420p -vf "fps=24" {left_eye_vid}
!ffmpeg -framerate 24 -i {right_eye_dir} -c:v libx264 -pix_fmt yuv420p -vf "fps=24" {right_eye_vid}



## Merge Videos and Inject Metadata

Combine the left and right eye videos into an SBS video and inject 3D metadata.


In [ ]:
left_eye_vid = dataset_directory + "left_eye.mp4"
right_eye_vid = dataset_directory + "right_eye.mp4"
output_vid = dataset_directory + "output.SBS.mp4"

!ffmpeg -i {left_eye_vid} -i {right_eye_vid} -filter_complex "[0:v][1:v]hstack=inputs=2[v]" -map "[v]" {output_vid}
!ffmpeg -i {output_vid} -vf "scale=2*iw:ih" -c:v libx264 -x264opts "frame-packing=3" -aspect 2:1 {final_video_filename}


## Upload SBS video to Quest headset

First register device connection, then push file, and lastly force the file system to update without restarting the device.


In [ ]:
!adb devices
!adb push {final_video_filename} /sdcard/Movies/
!adb shell am force-stop com.android.providers.media.module


## Cleanup and Finalization

(Optional) Cleanup temporary files and display/export the final video path.


In [ ]:
# Example cleanup (adjust as needed)

#rgbd frame directories
rgbd_frames = dataset_directory + 'rgbd_in/'
#stereo directories
stereo_input_dir =  dataset_directory + "input_frames/"
stereo_output_dir = dataset_directory + "stereo_out_frames/"
#inpainting directories
stereo_postprocess_dir = dataset_directory + "stereo_postprocess_frames/"

#video directories
left_eye_vid = dataset_directory + "left_eye.mp4"
right_eye_vid = dataset_directory + "right_eye.mp4"
output_vid = dataset_directory + "output.SBS.mp4"

# delete all directories and videos
!rm -rf {rgbd_frames}
!rm -rf {stereo_input_dir}
!rm -rf {stereo_output_dir}
!rm -rf {stereo_postprocess_dir}
!rm {left_eye_vid} {right_eye_vid} {output_vid}